Aca vamos a hacer la parte de generacion y evaluacion de modelos

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns 

from scripts import featurizer_xgb
from scripts import featurizer_rn
from scripts import featurizer_rf


from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_curve, recall_score, accuracy_score,precision_score ,f1_score

In [2]:
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier

In [3]:
from keras.optimizers import SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


2022-08-11 16:30:05.917055: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 16:30:05.917071: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, roc_auc_score


In [5]:
def XGBMaster(hiperparametros, X, Y):
    xgb = XGBClassifier()
    random_search_xgb = RandomizedSearchCV(xgb,
                                       param_distributions = hiperparametros,
                                       n_iter = 5, ##cambiar a mas
                                       scoring = 'roc_auc',
                                       cv = 5,
                                       n_jobs = -1,
                                       verbose=1)
    random_search_xgb.fit(X, Y)
    params_opt_xgb = random_search_xgb.best_params_
    
    return XGBClassifier(**params_opt_xgb)

In [6]:
def RFMaster(hiperparametros, X, Y):
    rf = RandomForestClassifier()#(random_state=1)##se puede cambiar
    grid_search_rf = GridSearchCV(estimator=rf, 
                                param_grid=hiperparametros, 
                                scoring="roc_auc", 
                                cv=5, 
                                n_jobs=-1,
                                verbose = 1)
    grid_search_rf.fit(X, Y)
    params_opt_rf = grid_search_rf.best_params_
    return RandomForestClassifier(**params_opt_rf)
    

In [7]:
def RNMaster(X, Y):
    
    opt = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)

    model = Sequential()
    model.add(Dense(8, input_dim=X.shape[1], activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    
    model.fit(X, Y, epochs=1000, verbose=1, batch_size=100,validation_split=0.2) ##epochs a cambiar
    return model

In [8]:
models = ['redneuronal','xgb','randomforest']
featurizers = [featurizer_rn, featurizer_xgb, featurizer_rf]
hiperparametros = {'xgb':{'n_estimators':[50, 100],
              'max_depth':[5,10],              
              'subsample': [0.5,0.8,1],
              'colsample_bytree': [0.5,0.8,1],              
              'learning_rate':[0.01, 0.1, 0.3]
              },
                  'randomforest': { "criterion" : ["gini", "entropy"], 
               "min_samples_leaf" : [1, 5], 
               "max_features":["sqrt", "log2"],
               "min_samples_split" : [2, 5], 
               "n_estimators": [10,50] }}

In [9]:
def entrenar_modelo_seleccionado(modelo, X, Y):
    if modelo == 'xgb':
        return XGBMaster(hiperparametros[modelo], X, Y)
    if modelo == 'randomforest':
        return RFMaster(hiperparametros[modelo], X, Y)
    return RNMaster(X.values,Y)

In [10]:
def print_metricas(prediccion, valor_real):
    print("auc = ", roc_auc_score(valor_real, prediccion))

In [11]:
def limpiar_datos_mal_predecidos(df, df_original):
    
    df['mal_predicho'] = df['valor_predicho'] == df['valor_real']
    df_mal_predichos = df[df['mal_predicho'] == 0]
    
    print("cantidad de elementos mal predichos {}".format(df_mal_predichos.shape[0]))
    
    return df_original.iloc[list(df_mal_predichos.index)]



In [12]:
def create_train_test_split(X, Y):
    kfold = StratifiedKFold(n_splits=5, shuffle=True)
    for train, test in kfold.split(X, y):
        X_train, X_test = X[train], X[test]
        y_train, y_test = y[train], y[test]
    return X_train, X_test, y_train, y_test

In [13]:
df = pd.read_csv("../data/dataset-procesado.csv", index_col=False)
df_copy = df

In [14]:
for i, model in enumerate(models):
    print("Elementos a fitear : {}".format(df_copy.shape[0]))
    print("Step number {} with model {}".format(i + 1, model))
    
    if (df_copy.shape[0] == 0):
        break 
    
    if (model == "redneuronal"):
        X, y = featurizer_rn.featurizer(df_copy)
    else:
        df_featurizado = featurizers[i].featurizer(df_copy)

        X = df_featurizado.drop(axis = 1, columns = ['target'])
        y = df_featurizado.target

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y)
    
    model = entrenar_modelo_seleccionado(model, X_train, y_train)
    
    print(model)
    
    
    if (model != "redneuronal"):
        model.fit(X_train, y_train)
    
    training = np.round(model.predict(X_train))
    testing = np.round(model.predict(X_test))
        
    
    print_metricas(training, y_train)
    print_metricas(testing, y_test)
    
    X_train['valor_predicho'] = training
    X_train['valor_real'] = y_train
    
    X_test['valor_predicho'] = testing
    X_test['valor_real'] = y_test
    
    df_limpio_train = limpiar_datos_mal_predecidos(X_train, df)
    df_limpio_test = limpiar_datos_mal_predecidos(X_test, df)

    frames = [df_limpio_train, df_limpio_test]
    df_copy = pd.concat(frames)
    
    
    print("----------------------------------------------------------------")
    
    
    
    
    

Elementos a fitear : 251091
Step number 1 with model redneuronal
----rn-featurizer----
Epoch 1/1000


2022-08-11 16:30:13.743095: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-11 16:30:13.743554: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-11 16:30:13.743589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-08-11 16:30:13.743621: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-08-11 16:30:13.743652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

1607/1607 [==============================] - 2s 1ms/step - loss: 0.3341 - accuracy: 0.8458 - val_loss: 0.2881 - val_accuracy: 0.8622
Epoch 2/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2916 - accuracy: 0.8644 - val_loss: 0.2826 - val_accuracy: 0.8661
Epoch 3/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2877 - accuracy: 0.8667 - val_loss: 0.2814 - val_accuracy: 0.8677
Epoch 4/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2865 - accuracy: 0.8666 - val_loss: 0.2807 - val_accuracy: 0.8685
Epoch 5/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2853 - accuracy: 0.8673 - val_loss: 0.2824 - val_accuracy: 0.8687
Epoch 6/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2842 - accuracy: 0.8678 - val_loss: 0.2789 - val_accuracy: 0.8686
Epoch 7/1000
1607/1607 [==============================] - 2s 1ms/step - loss: 0.2836 - accuracy: 0.8679 - val_loss: 0.2786 - val_accuracy